In [24]:
import sqlite3
import pandas as pd
from typing import Optional
from datetime import datetime

def query_candles(
    db_path: str = 'hl_candles.db',
    exchange_name: Optional[str] = None,
    coin_symbol: Optional[str] = None,
    interval: Optional[str] = None,
    start_time: Optional[int] = None,
    end_time: Optional[int] = None,
    limit: Optional[int] = None,
    order: str = 'asc'
) -> pd.DataFrame:
    if order.lower() not in ['asc', 'desc']:
        raise ValueError("Parameter 'order' must be either 'asc' or 'desc'.")

    try:
        conn = sqlite3.connect(db_path)
    except sqlite3.Error as e:
        raise ConnectionError(f"Unable to connect to the database: {e}")

    query = """
    SELECT 
        e.name AS exchange,
        c.symbol AS coin,
        i.int AS interval,
        ca.s_t,
        ca.e_t,
        ca.o,
        ca.c,
        ca.h,
        ca.l,
        ca.v,
        ca.n
    FROM candles ca
    JOIN exchanges e ON ca.exchange_id = e.exchange_id
    JOIN coins c ON ca.coin_id = c.coin_id
    JOIN intervals i ON ca.interval_id = i.interval_id
    WHERE 1=1
    """
    params = []
    if exchange_name:
        query += " AND e.name = ?"
        params.append(exchange_name)
    if coin_symbol:
        query += " AND c.symbol = ?"
        params.append(coin_symbol)
    if interval:
        query += " AND i.int = ?"
        params.append(interval)
    if start_time:
        query += " AND ca.s_t >= ?"
        params.append(start_time)
    if end_time:
        query += " AND ca.e_t <= ?"
        params.append(end_time)

    query += f" ORDER BY ca.s_t {order.upper()}"

    if limit:
        query += " LIMIT ?"
        params.append(limit)

    try:
        df = pd.read_sql_query(query, conn, params=params)
    except pd.io.sql.DatabaseError as e:
        raise RuntimeError(f"An error occurred while executing the query: {e}")
    finally:
        conn.close()

    return df

In [9]:
resp = query_candles('../candle_data/candle_db/candles.db', exchange_name='Hyperliquid', coin_symbol='AAVE', interval='1m')

In [10]:
resp

,exchange,coin,interval,s_t,e_t,o,c,h,l,v,n
0,Hyperliquid,AAVE,1m,1737123900000,1737123959999,328.32,328.61,328.84,328.32,32.05,18
1,Hyperliquid,AAVE,1m,1737123960000,1737124019999,328.61,328.55,329.05,328.55,8.57,8
2,Hyperliquid,AAVE,1m,1737124020000,1737124079999,328.50,328.32,328.50,328.13,9.75,10
3,Hyperliquid,AAVE,1m,1737124080000,1737124139999,328.37,328.33,328.45,328.10,23.50,19
4,Hyperliquid,AAVE,1m,1737124140000,1737124199999,328.33,328.45,328.84,328.33,28.29,17
...,...,...,...,...,...,...,...,...,...,...,...
6254,Hyperliquid,AAVE,1m,1737499140000,1737499199999,370.61,370.19,370.65,370.00,78.17,51
6255,Hyperliquid,AAVE,1m,1737499200000,1737499259999,370.01,369.50,370.03,369.30,46.04,31
6256,Hyperliquid,AAVE,1m,1737499260000,1737499319999,369.50,369.31,369.50,369.26,36.00,17
6257,Hyperliquid,AAVE,1m,1737499320000,1737499379999,369.31,369.43,369.43,369.31,12.75,21


In [23]:
for idx, candle in resp.iterrows():
    print(candle['s_t'])

1737123900000
1737123960000
1737124020000
1737124080000
1737124140000
1737124200000
1737124260000
1737124320000
1737124380000
1737124440000
1737124500000
1737124560000
1737124620000
1737124680000
1737124740000
1737124800000
1737124860000
1737124920000
1737124980000
1737125040000
1737125100000
1737125160000
1737125220000
1737125280000
1737125340000
1737125400000
1737125460000
1737125520000
1737125580000
1737125640000
1737125700000
1737125760000
1737125820000
1737125880000
1737125940000
1737126000000
1737126060000
1737126120000
1737126180000
1737126240000
1737126300000
1737126360000
1737126420000
1737126480000
1737126540000
1737126600000
1737126660000
1737126720000
1737126780000
1737126840000
1737126900000
1737126960000
1737127020000
1737127080000
1737127140000
1737127200000
1737127260000
1737127320000
1737127380000
1737127440000
1737127500000
1737127560000
1737127620000
1737127680000
1737127740000
1737127800000
1737127860000
1737127920000
1737127980000
1737128040000
1737128100000
173712

In [13]:
formatted = [{
            's_t': int(candle['s_t'])
        } for candle in resp.to_l]
formatted

TypeError: 'method' object is not iterable

In [43]:
int_dict = resp.groupby('interval').apply(lambda x: x.to_dict(orient='records')).to_dict()

In [49]:
sorted_interval = int_dict['1m'].sort(key=lambda x: x['s_t'])

In [51]:
sorted_interval = sorted(int_dict['1m'], key=lambda x: x['s_t'])
sorted_interval

[{'exchange': 'Hyperliquid',
  'coin': 'AAVE',
  'interval': '1m',
  's_t': 1737123900000,
  'e_t': 1737123959999,
  'o': 328.32,
  'c': 328.61,
  'h': 328.84,
  'l': 328.32,
  'v': 32.05,
  'n': 18},
 {'exchange': 'Hyperliquid',
  'coin': 'AAVE',
  'interval': '1m',
  's_t': 1737123960000,
  'e_t': 1737124019999,
  'o': 328.61,
  'c': 328.55,
  'h': 329.05,
  'l': 328.55,
  'v': 8.57,
  'n': 8},
 {'exchange': 'Hyperliquid',
  'coin': 'AAVE',
  'interval': '1m',
  's_t': 1737124020000,
  'e_t': 1737124079999,
  'o': 328.5,
  'c': 328.32,
  'h': 328.5,
  'l': 328.13,
  'v': 9.75,
  'n': 10},
 {'exchange': 'Hyperliquid',
  'coin': 'AAVE',
  'interval': '1m',
  's_t': 1737124080000,
  'e_t': 1737124139999,
  'o': 328.37,
  'c': 328.33,
  'h': 328.45,
  'l': 328.1,
  'v': 23.5,
  'n': 19},
 {'exchange': 'Hyperliquid',
  'coin': 'AAVE',
  'interval': '1m',
  's_t': 1737124140000,
  'e_t': 1737124199999,
  'o': 328.33,
  'c': 328.45,
  'h': 328.84,
  'l': 328.33,
  'v': 28.29,
  'n': 17},
 {

In [52]:
sorted_interval[-1]

{'exchange': 'Hyperliquid',
 'coin': 'AAVE',
 'interval': '1m',
 's_t': 1737433260000,
 'e_t': 1737433319999,
 'o': 341.66,
 'c': 341.24,
 'h': 341.66,
 'l': 341.24,
 'v': 13.49,
 'n': 11}

In [2]:
local_candles = {'hl': {'AAVE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'AAVE', 'i': '1m', 'o': '345.58', 'c': '345.34', 'h': '345.58', 'l': '345.07', 'v': '3.35', 'n': 5}], 'ACE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ACE', 'i': '1m', 'o': '1.5314', 'c': '1.5314', 'h': '1.5314', 'l': '1.5314', 'v': '11.25', 'n': 1}], 'ADA': [], 'AI': [{'t': 1737431460000, 'T': 1737431519999, 's': 'AI', 'i': '1m', 'o': '0.56202', 'c': '0.56166', 'h': '0.56202', 'l': '0.56166', 'v': '69.3', 'n': 2}], 'AI16Z': [], 'AIXBT': [{'t': 1737431460000, 'T': 1737431519999, 's': 'AIXBT', 'i': '1m', 'o': '0.65893', 'c': '0.65776', 'h': '0.65898', 'l': '0.65776', 'v': '5833.0', 'n': 13}], 'ALGO': [], 'ALT': [], 'APE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'APE', 'i': '1m', 'o': '0.9671', 'c': '0.9671', 'h': '0.9671', 'l': '0.9671', 'v': '18.5', 'n': 1}], 'APT': [], 'AR': [], 'ARB': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ARB', 'i': '1m', 'o': '0.68404', 'c': '0.68404', 'h': '0.68404', 'l': '0.68404', 'v': '23.4', 'n': 1}], 'ATOM': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ATOM', 'i': '1m', 'o': '5.9663', 'c': '5.9663', 'h': '5.9663', 'l': '5.9663', 'v': '8.18', 'n': 1}], 'AVAX': [{'t': 1737431460000, 'T': 1737431519999, 's': 'AVAX', 'i': '1m', 'o': '35.203', 'c': '35.203', 'h': '35.203', 'l': '35.203', 'v': '6.57', 'n': 1}], 'BADGER': [], 'BANANA': [{'t': 1737431460000, 'T': 1737431519999, 's': 'BANANA', 'i': '1m', 'o': '35.816', 'c': '35.816', 'h': '35.816', 'l': '35.816', 'v': '0.5', 'n': 1}], 'BIGTIME': [], 'BIO': [], 'BLAST': [{'t': 1737431460000, 'T': 1737431519999, 's': 'BLAST', 'i': '1m', 'o': '0.007524', 'c': '0.007524', 'h': '0.007524', 'l': '0.007524', 'v': '5457.0', 'n': 1}], 'BLUR': [], 'BLZ': [], 'BNB': [], 'BRETT': [], 'BSV': [], 'BTC': [{'t': 1737431460000, 'T': 1737431519999, 's': 'BTC', 'i': '1m', 'o': '103141.0', 'c': '103078.0', 'h': '103143.0', 'l': '103065.0', 'v': '4.05143', 'n': 133}], 'CAKE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'CAKE', 'i': '1m', 'o': '2.1196', 'c': '2.1196', 'h': '2.1196', 'l': '2.1196', 'v': '8.3', 'n': 1}], 'CANTO': [], 'CATI': [{'t': 1737431460000, 'T': 1737431519999, 's': 'CATI', 'i': '1m', 'o': '0.24442', 'c': '0.24442', 'h': '0.24442', 'l': '0.24442', 'v': '81.0', 'n': 1}], 'CELO': [{'t': 1737431460000, 'T': 1737431519999, 's': 'CELO', 'i': '1m', 'o': '0.57875', 'c': '0.57875', 'h': '0.57875', 'l': '0.57875', 'v': '33.0', 'n': 1}], 'CHILLGUY': [{'t': 1737431460000, 'T': 1737431519999, 's': 'CHILLGUY', 'i': '1m', 'o': '0.073152', 'c': '0.073152', 'h': '0.073152', 'l': '0.073152', 'v': '217.0', 'n': 1}], 'COMP': [], 'DYDX': [], 'DYM': [{'t': 1737431460000, 'T': 1737431519999, 's': 'DYM', 'i': '1m', 'o': '1.0852', 'c': '1.0847', 'h': '1.0853', 'l': '1.0847', 'v': '68.0', 'n': 4}], 'EIGEN': [], 'ENS': [], 'ETC': [], 'ETH': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ETH', 'i': '1m', 'o': '3265.0', 'c': '3262.2', 'h': '3265.0', 'l': '3262.1', 'v': '210.7538', 'n': 60}], 'FARTCOIN': [{'t': 1737431460000, 'T': 1737431519999, 's': 'FARTCOIN', 'i': '1m', 'o': '1.788', 'c': '1.7925', 'h': '1.7925', 'l': '1.7878', 'v': '1134.2', 'n': 9}], 'FET': [], 'FIL': [], 'FRIEND': [], 'FTM': [], 'FTT': [{'t': 1737431460000, 'T': 1737431519999, 's': 'FTT', 'i': '1m', 'o': '1.8597', 'c': '1.8597', 'h': '1.8597', 'l': '1.8597', 'v': '10.7', 'n': 1}], 'FXS': [{'t': 1737431460000, 'T': 1737431519999, 's': 'FXS', 'i': '1m', 'o': '2.9171', 'c': '2.9171', 'h': '2.9171', 'l': '2.9171', 'v': '6.6', 'n': 1}], 'GALA': [], 'GAS': [{'t': 1737431460000, 'T': 1737431519999, 's': 'GAS', 'i': '1m', 'o': '4.7318', 'c': '4.7318', 'h': '4.7318', 'l': '4.7318', 'v': '3.8', 'n': 1}], 'GMT': [{'t': 1737431460000, 'T': 1737431519999, 's': 'GMT', 'i': '1m', 'o': '0.098684', 'c': '0.098684', 'h': '0.098684', 'l': '0.098684', 'v': '177.0', 'n': 1}], 'GOAT': [], 'GRASS': [{'t': 1737431460000, 'T': 1737431519999, 's': 'GRASS', 'i': '1m', 'o': '1.9516', 'c': '1.9516', 'h': '1.9516', 'l': '1.9516', 'v': '8.1', 'n': 1}], 'GRIFFAIN': [{'t': 1737431460000, 'T': 1737431519999, 's': 'GRIFFAIN', 'i': '1m', 'o': '0.49922', 'c': '0.49743', 'h': '0.49922', 'l': '0.49732', 'v': '11155.0', 'n': 8}], 'HBAR': [{'t': 1737431460000, 'T': 1737431519999, 's': 'HBAR', 'i': '1m', 'o': '0.33579', 'c': '0.33558', 'h': '0.33579', 'l': '0.33558', 'v': '1802.0', 'n': 5}], 'HMSTR': [], 'HPOS': [], 'HYPE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'HYPE', 'i': '1m', 'o': '21.171', 'c': '21.169', 'h': '21.172', 'l': '21.168', 'v': '38.42', 'n': 8}], 'ILV': [], 'IMX': [], 'INJ': [{'t': 1737431460000, 'T': 1737431519999, 's': 'INJ', 'i': '1m', 'o': '20.644', 'c': '20.644', 'h': '20.644', 'l': '20.644', 'v': '1.4', 'n': 1}], 'IO': [], 'IOTA': [{'t': 1737431460000, 'T': 1737431519999, 's': 'IOTA', 'i': '1m', 'o': '0.31762', 'c': '0.31762', 'h': '0.31762', 'l': '0.31762', 'v': '51.0', 'n': 1}], 'JTO': [{'t': 1737431460000, 'T': 1737431519999, 's': 'JTO', 'i': '1m', 'o': '2.8518', 'c': '2.8518', 'h': '2.8518', 'l': '2.8518', 'v': '61.0', 'n': 1}], 'JUP': [{'t': 1737431460000, 'T': 1737431519999, 's': 'JUP', 'i': '1m', 'o': '0.9005', 'c': '0.9005', 'h': '0.9005', 'l': '0.9005', 'v': '20.0', 'n': 1}], 'KAS': [], 'LDO': [], 'LISTA': [], 'MANTA': [], 'MATIC': [], 'MAV': [], 'MAVIA': [], 'ME': [], 'MELANIA': [{'t': 1737431460000, 'T': 1737431519999, 's': 'MELANIA', 'i': '1m', 'o': '4.08', 'c': '4.0531', 'h': '4.0825', 'l': '4.053', 'v': '20808.3', 'n': 90}], 'MEME': [], 'MERL': [], 'MEW': [], 'MINA': [], 'MKR': [], 'MOODENG': [{'t': 1737431460000, 'T': 1737431519999, 's': 'MOODENG', 'i': '1m', 'o': '0.14068', 'c': '0.14067', 'h': '0.14068', 'l': '0.14067', 'v': '235.0', 'n': 2}], 'MORPHO': [], 'MOVE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'MOVE', 'i': '1m', 'o': '0.68826', 'c': '0.68826', 'h': '0.68826', 'l': '0.68826', 'v': '663.0', 'n': 3}], 'NEAR': [], 'NEIROETH': [], 'NEO': [], 'NFTI': [], 'NOT': [], 'NTRN': [], 'OMNI': [], 'ONDO': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ONDO', 'i': '1m', 'o': '1.2782', 'c': '1.2781', 'h': '1.2782', 'l': '1.2781', 'v': '1500.0', 'n': 2}], 'OP': [], 'ORDI': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ORDI', 'i': '1m', 'o': '18.636', 'c': '18.636', 'h': '18.636', 'l': '18.636', 'v': '1.78', 'n': 1}], 'OX': [], 'PANDORA': [], 'PENDLE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'PENDLE', 'i': '1m', 'o': '3.5823', 'c': '3.5823', 'h': '3.5823', 'l': '3.5823', 'v': '485.0', 'n': 3}], 'PENGU': [{'t': 1737431460000, 'T': 1737431519999, 's': 'PENGU', 'i': '1m', 'o': '0.023969', 'c': '0.023969', 'h': '0.023969', 'l': '0.023969', 'v': '24884.0', 'n': 5}], 'PEOPLE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'PEOPLE', 'i': '1m', 'o': '0.029436', 'c': '0.029436', 'h': '0.029436', 'l': '0.029436', 'v': '1241.0', 'n': 1}], 'PIXEL': [], 'PNUT': [], 'POL': [], 'POLYX': [], 'POPCAT': [{'t': 1737431460000, 'T': 1737431519999, 's': 'POPCAT', 'i': '1m', 'o': '0.44136', 'c': '0.44078', 'h': '0.44136', 'l': '0.44078', 'v': '2269.0', 'n': 2}], 'PURR': [], 'PURR/USDC': [], 'PYTH': [], 'RENDER': [{'t': 1737431460000, 'T': 1737431519999, 's': 'RENDER', 'i': '1m', 'o': '6.5926', 'c': '6.5832', 'h': '6.5926', 'l': '6.5832', 'v': '9.1', 'n': 2}], 'REQ': [], 'REZ': [{'t': 1737431460000, 'T': 1737431519999, 's': 'REZ', 'i': '1m', 'o': '0.028637', 'c': '0.028637', 'h': '0.028637', 'l': '0.028637', 'v': '1087.0', 'n': 1}], 'RLB': [], 'RNDR': [], 'RSR': [{'t': 1737431460000, 'T': 1737431519999, 's': 'RSR', 'i': '1m', 'o': '0.013026', 'c': '0.013026', 'h': '0.013026', 'l': '0.013026', 'v': '1323.0', 'n': 1}], 'RUNE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'RUNE', 'i': '1m', 'o': '3.2', 'c': '3.2', 'h': '3.2', 'l': '3.2', 'v': '202.0', 'n': 1}], 'S': [{'t': 1737431460000, 'T': 1737431519999, 's': 'S', 'i': '1m', 'o': '0.51688', 'c': '0.51688', 'h': '0.51688', 'l': '0.51688', 'v': '45.0', 'n': 1}], 'SAGA': [{'t': 1737431460000, 'T': 1737431519999, 's': 'SAGA', 'i': '1m', 'o': '1.2682', 'c': '1.2674', 'h': '1.2682', 'l': '1.2674', 'v': '41.7', 'n': 2}], 'SAND': [], 'SCR': [{'t': 1737431460000, 'T': 1737431519999, 's': 'SCR', 'i': '1m', 'o': '0.75346', 'c': '0.75346', 'h': '0.75346', 'l': '0.75346', 'v': '252.9', 'n': 1}], 'SNX': [], 'SOL': [{'t': 1737431460000, 'T': 1737431519999, 's': 'SOL', 'i': '1m', 'o': '238.05', 'c': '237.9', 'h': '238.05', 'l': '237.71', 'v': '157.98', 'n': 29}], 'SPX': [{'t': 1737431460000, 'T': 1737431519999, 's': 'SPX', 'i': '1m', 'o': '1.378', 'c': '1.3763', 'h': '1.378', 'l': '1.3762', 'v': '714.1', 'n': 5}], 'STG': [{'t': 1737431460000, 'T': 1737431519999, 's': 'STG', 'i': '1m', 'o': '0.33584', 'c': '0.33584', 'h': '0.33584', 'l': '0.33584', 'v': '54.0', 'n': 1}], 'STRAX': [], 'STX': [{'t': 1737431460000, 'T': 1737431519999, 's': 'STX', 'i': '1m', 'o': '1.3831', 'c': '1.383', 'h': '1.3831', 'l': '1.383', 'v': '43.1', 'n': 2}], 'SUI': [{'t': 1737431460000, 'T': 1737431519999, 's': 'SUI', 'i': '1m', 'o': '4.3437', 'c': '4.3387', 'h': '4.3437', 'l': '4.3381', 'v': '122.4', 'n': 3}], 'SUPER': [], 'SUSHI': [], 'TAO': [{'t': 1737431460000, 'T': 1737431519999, 's': 'TAO', 'i': '1m', 'o': '397.81', 'c': '397.68', 'h': '397.81', 'l': '397.67', 'v': '24.74', 'n': 7}], 'TIA': [{'t': 1737431460000, 'T': 1737431519999, 's': 'TIA', 'i': '1m', 'o': '4.5027', 'c': '4.5027', 'h': '4.5027', 'l': '4.5027', 'v': '3.4', 'n': 1}], 'TNSR': [], 'TON': [], 'TRB': [], 'TRUMP': [{'t': 1737431460000, 'T': 1737431519999, 's': 'TRUMP', 'i': '1m', 'o': '35.253', 'c': '35.211', 'h': '35.371', 'l': '35.211', 'v': '2953.8', 'n': 160}], 'TURBO': [{'t': 1737431460000, 'T': 1737431519999, 's': 'TURBO', 'i': '1m', 'o': '0.006405', 'c': '0.006398', 'h': '0.006405', 'l': '0.006398', 'v': '6976.0', 'n': 2}], 'UMA': [{'t': 1737431460000, 'T': 1737431519999, 's': 'UMA', 'i': '1m', 'o': '2.4277', 'c': '2.4277', 'h': '2.4277', 'l': '2.4277', 'v': '15.5', 'n': 1}], 'UNIBOT': [], 'USTC': [], 'USUAL': [], 'VIRTUAL': [{'t': 1737431460000, 'T': 1737431519999, 's': 'VIRTUAL', 'i': '1m', 'o': '2.543', 'c': '2.543', 'h': '2.543', 'l': '2.543', 'v': '187.4', 'n': 1}], 'W': [], 'WIF': [{'t': 1737431460000, 'T': 1737431519999, 's': 'WIF', 'i': '1m', 'o': '1.3448', 'c': '1.3446', 'h': '1.3448', 'l': '1.3446', 'v': '5003.0', 'n': 2}], 'WLD': [], 'XAI': [], 'XLM': [{'t': 1737431460000, 'T': 1737431519999, 's': 'XLM', 'i': '1m', 'o': '0.43961', 'c': '0.43961', 'h': '0.43961', 'l': '0.43961', 'v': '58.0', 'n': 1}], 'ZEN': [], 'ZEREBRO': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ZEREBRO', 'i': '1m', 'o': '0.14002', 'c': '0.14002', 'h': '0.14002', 'l': '0.14002', 'v': '2558.0', 'n': 2}], 'ZETA': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ZETA', 'i': '1m', 'o': '0.46253', 'c': '0.46243', 'h': '0.46253', 'l': '0.46243', 'v': '71.8', 'n': 2}], 'ZK': [], 'ZRO': [{'t': 1737431460000, 'T': 1737431519999, 's': 'ZRO', 'i': '1m', 'o': '3.9923', 'c': '3.9913', 'h': '3.9923', 'l': '3.9908', 'v': '4000.0', 'n': 9}], 'kBONK': [{'t': 1737431460000, 'T': 1737431519999, 's': 'kBONK', 'i': '1m', 'o': '0.029269', 'c': '0.029245', 'h': '0.029269', 'l': '0.029245', 'v': '132552.0', 'n': 2}], 'kDOGS': [], 'kLUNC': [{'t': 1737431460000, 'T': 1737431519999, 's': 'kLUNC', 'i': '1m', 'o': '0.084541', 'c': '0.084523', 'h': '0.084541', 'l': '0.084512', 'v': '766.0', 'n': 3}], 'kNEIRO': [], 'kPEPE': [{'t': 1737431460000, 'T': 1737431519999, 's': 'kPEPE', 'i': '1m', 'o': '0.015349', 'c': '0.015327', 'h': '0.015349', 'l': '0.015327', 'v': '348584.0', 'n': 7}], 'kSHIB': [{'t': 1737431460000, 'T': 1737431519999, 's': 'kSHIB', 'i': '1m', 'o': '0.020053', 'c': '0.020053', 'h': '0.020053', 'l': '0.020053', 'v': '919.0', 'n': 1}]}}

In [3]:
local_coin_candles = local_candles['hl']['AAVE']
local_coin_candles

[{'t': 1737431460000,
  'T': 1737431519999,
  's': 'AAVE',
  'i': '1m',
  'o': '345.58',
  'c': '345.34',
  'h': '345.58',
  'l': '345.07',
  'v': '3.35',
  'n': 5}]

In [5]:
local_coin_candles[-1]['T']

{'t': 1737431460000,
 'T': 1737431519999,
 's': 'AAVE',
 'i': '1m',
 'o': '345.58',
 'c': '345.34',
 'h': '345.58',
 'l': '345.07',
 'v': '3.35',
 'n': 5}

In [25]:
old = [{'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516600000, 'e_t': 1737516659999, 's': 'AAVE', 'i': '1m', 'o': 365.17, 'c': 364.93, 'h': 365.18, 'l': 364.93, 'v': 5.61, 'n': 12}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516660000, 'e_t': 1737516719999, 's': 'AAVE', 'i': '1m', 'o': 364.88, 'c': 365.33, 'h': 365.49, 'l': 364.37, 'v': 13.86, 'n': 19}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516720000, 'e_t': 1737516779999, 's': 'AAVE', 'i': '1m', 'o': 365.69, 'c': 366.78, 'h': 367.65, 'l': 365.69, 'v': 207.95, 'n': 77}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516780000, 'e_t': 1737516839999, 's': 'AAVE', 'i': '1m', 'o': 366.74, 'c': 366.48, 'h': 366.74, 'l': 366.48, 'v': 1.09, 'n': 4}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516840000, 'e_t': 1737516899999, 's': 'AAVE', 'i': '1m', 'o': 365.72, 'c': 365.6, 'h': 365.84, 'l': 365.5, 'v': 7.56, 'n': 11}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516900000, 'e_t': 1737516959999, 's': 'AAVE', 'i': '1m', 'o': 365.95, 'c': 366.03, 'h': 366.03, 'l': 365.72, 'v': 9.24, 'n': 17}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516960000, 'e_t': 1737517019999, 's': 'AAVE', 'i': '1m', 'o': 365.97, 'c': 365.89, 'h': 365.97, 'l': 365.89, 'v': 2.83, 'n': 4}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737517020000, 'e_t': 1737517079999, 's': 'AAVE', 'i': '1m', 'o': 366.03, 'c': 365.71, 'h': 366.27, 'l': 365.71, 'v': 3.59, 'n': 10}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737517080000, 'e_t': 1737517139999, 's': 'AAVE', 'i': '1m', 'o': 365.7, 'c': 365.69, 'h': 365.7, 'l': 365.69, 'v': 0.44, 'n': 3}, {'s_t': 1737517140000, 'e_t': 1737517199999, 's': 'AAVE', 'i': '1m', 'o': '366.73', 'c': '367.65', 'h': '367.66', 'l': '366.6', 'v': '21.96', 'n': 26}]
old_df = pd.DataFrame(old)
old_df

,exchange,coin,interval,s_t,e_t,s,i,o,c,h,l,v,n
0,Hyperliquid,AAVE,1m,1737516600000,1737516659999,AAVE,1m,365.17,364.93,365.18,364.93,5.61,12
1,Hyperliquid,AAVE,1m,1737516660000,1737516719999,AAVE,1m,364.88,365.33,365.49,364.37,13.86,19
2,Hyperliquid,AAVE,1m,1737516720000,1737516779999,AAVE,1m,365.69,366.78,367.65,365.69,207.95,77
3,Hyperliquid,AAVE,1m,1737516780000,1737516839999,AAVE,1m,366.74,366.48,366.74,366.48,1.09,4
4,Hyperliquid,AAVE,1m,1737516840000,1737516899999,AAVE,1m,365.72,365.6,365.84,365.5,7.56,11
5,Hyperliquid,AAVE,1m,1737516900000,1737516959999,AAVE,1m,365.95,366.03,366.03,365.72,9.24,17
6,Hyperliquid,AAVE,1m,1737516960000,1737517019999,AAVE,1m,365.97,365.89,365.97,365.89,2.83,4
7,Hyperliquid,AAVE,1m,1737517020000,1737517079999,AAVE,1m,366.03,365.71,366.27,365.71,3.59,10
8,Hyperliquid,AAVE,1m,1737517080000,1737517139999,AAVE,1m,365.7,365.69,365.7,365.69,0.44,3
9,NaN,NaN,NaN,1737517140000,1737517199999,AAVE,1m,366.73,367.65,367.66,366.6,21.96,26


In [26]:
new = [{'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516660000, 'e_t': 1737516719999, 's': 'AAVE', 'i': '1m', 'o': 364.88, 'c': 365.33, 'h': 365.49, 'l': 364.37, 'v': 13.86, 'n': 19}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516720000, 'e_t': 1737516779999, 's': 'AAVE', 'i': '1m', 'o': 365.69, 'c': 366.78, 'h': 367.65, 'l': 365.69, 'v': 207.95, 'n': 77}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516780000, 'e_t': 1737516839999, 's': 'AAVE', 'i': '1m', 'o': 366.74, 'c': 366.48, 'h': 366.74, 'l': 366.48, 'v': 1.09, 'n': 4}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516840000, 'e_t': 1737516899999, 's': 'AAVE', 'i': '1m', 'o': 365.72, 'c': 365.6, 'h': 365.84, 'l': 365.5, 'v': 7.56, 'n': 11}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516900000, 'e_t': 1737516959999, 's': 'AAVE', 'i': '1m', 'o': 365.95, 'c': 366.03, 'h': 366.03, 'l': 365.72, 'v': 9.24, 'n': 17}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516960000, 'e_t': 1737517019999, 's': 'AAVE', 'i': '1m', 'o': 365.97, 'c': 365.89, 'h': 365.97, 'l': 365.89, 'v': 2.83, 'n': 4}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737517020000, 'e_t': 1737517079999, 's': 'AAVE', 'i': '1m', 'o': 366.03, 'c': 365.71, 'h': 366.27, 'l': 365.71, 'v': 3.59, 'n': 10}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737517080000, 'e_t': 1737517139999, 's': 'AAVE', 'i': '1m', 'o': 365.7, 'c': 365.69, 'h': 365.7, 'l': 365.69, 'v': 0.44, 'n': 3}, {'s_t': 1737517140000, 'e_t': 1737517199999, 's': 'AAVE', 'i': '1m', 'o': '366.73', 'c': '367.65', 'h': '367.66', 'l': '366.6', 'v': '21.96', 'n': 26}, {'s_t': 1737517200000, 'e_t': 1737517259999, 's': 'AAVE', 'i': '1m', 'o': '367.66', 'c': '367.66', 'h': '367.66', 'l': '367.66', 'v': '1.4', 'n': 1}]
new_df = pd.DataFrame(new)
new_df

,exchange,coin,interval,s_t,e_t,s,i,o,c,h,l,v,n
0,Hyperliquid,AAVE,1m,1737516660000,1737516719999,AAVE,1m,364.88,365.33,365.49,364.37,13.86,19
1,Hyperliquid,AAVE,1m,1737516720000,1737516779999,AAVE,1m,365.69,366.78,367.65,365.69,207.95,77
2,Hyperliquid,AAVE,1m,1737516780000,1737516839999,AAVE,1m,366.74,366.48,366.74,366.48,1.09,4
3,Hyperliquid,AAVE,1m,1737516840000,1737516899999,AAVE,1m,365.72,365.6,365.84,365.5,7.56,11
4,Hyperliquid,AAVE,1m,1737516900000,1737516959999,AAVE,1m,365.95,366.03,366.03,365.72,9.24,17
5,Hyperliquid,AAVE,1m,1737516960000,1737517019999,AAVE,1m,365.97,365.89,365.97,365.89,2.83,4
6,Hyperliquid,AAVE,1m,1737517020000,1737517079999,AAVE,1m,366.03,365.71,366.27,365.71,3.59,10
7,Hyperliquid,AAVE,1m,1737517080000,1737517139999,AAVE,1m,365.7,365.69,365.7,365.69,0.44,3
8,NaN,NaN,NaN,1737517140000,1737517199999,AAVE,1m,366.73,367.65,367.66,366.6,21.96,26
9,NaN,NaN,NaN,1737517200000,1737517259999,AAVE,1m,367.66,367.66,367.66,367.66,1.4,1


In [27]:
newest =[{'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516660000, 'e_t': 1737516719999, 's': 'AAVE', 'i': '1m', 'o': 364.88, 'c': 365.33, 'h': 365.49, 'l': 364.37, 'v': 13.86, 'n': 19}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516720000, 'e_t': 1737516779999, 's': 'AAVE', 'i': '1m', 'o': 365.69, 'c': 366.78, 'h': 367.65, 'l': 365.69, 'v': 207.95, 'n': 77}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516780000, 'e_t': 1737516839999, 's': 'AAVE', 'i': '1m', 'o': 366.74, 'c': 366.48, 'h': 366.74, 'l': 366.48, 'v': 1.09, 'n': 4}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516840000, 'e_t': 1737516899999, 's': 'AAVE', 'i': '1m', 'o': 365.72, 'c': 365.6, 'h': 365.84, 'l': 365.5, 'v': 7.56, 'n': 11}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516900000, 'e_t': 1737516959999, 's': 'AAVE', 'i': '1m', 'o': 365.95, 'c': 366.03, 'h': 366.03, 'l': 365.72, 'v': 9.24, 'n': 17}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737516960000, 'e_t': 1737517019999, 's': 'AAVE', 'i': '1m', 'o': 365.97, 'c': 365.89, 'h': 365.97, 'l': 365.89, 'v': 2.83, 'n': 4}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737517020000, 'e_t': 1737517079999, 's': 'AAVE', 'i': '1m', 'o': 366.03, 'c': 365.71, 'h': 366.27, 'l': 365.71, 'v': 3.59, 'n': 10}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737517080000, 'e_t': 1737517139999, 's': 'AAVE', 'i': '1m', 'o': 365.7, 'c': 365.69, 'h': 365.7, 'l': 365.69, 'v': 0.44, 'n': 3}, {'exchange': 'Hyperliquid', 'coin': 'AAVE', 'interval': '1m', 's_t': 1737517140000, 'e_t': 1737517199999, 's': 'AAVE', 'i': '1m', 'o': 366.73, 'c': 366.94, 'h': 366.94, 'l': 366.73, 'v': 16.61, 'n': 13}, {'s_t': 1737517200000, 'e_t': 1737517259999, 's': 'AAVE', 'i': '1m', 'o': '367.66', 'c': '367.66', 'h': '367.66', 'l': '367.66', 'v': '1.4', 'n': 1}]
newest_df = pd.DataFrame(newest)
newest_df

,exchange,coin,interval,s_t,e_t,s,i,o,c,h,l,v,n
0,Hyperliquid,AAVE,1m,1737516660000,1737516719999,AAVE,1m,364.88,365.33,365.49,364.37,13.86,19
1,Hyperliquid,AAVE,1m,1737516720000,1737516779999,AAVE,1m,365.69,366.78,367.65,365.69,207.95,77
2,Hyperliquid,AAVE,1m,1737516780000,1737516839999,AAVE,1m,366.74,366.48,366.74,366.48,1.09,4
3,Hyperliquid,AAVE,1m,1737516840000,1737516899999,AAVE,1m,365.72,365.6,365.84,365.5,7.56,11
4,Hyperliquid,AAVE,1m,1737516900000,1737516959999,AAVE,1m,365.95,366.03,366.03,365.72,9.24,17
5,Hyperliquid,AAVE,1m,1737516960000,1737517019999,AAVE,1m,365.97,365.89,365.97,365.89,2.83,4
6,Hyperliquid,AAVE,1m,1737517020000,1737517079999,AAVE,1m,366.03,365.71,366.27,365.71,3.59,10
7,Hyperliquid,AAVE,1m,1737517080000,1737517139999,AAVE,1m,365.7,365.69,365.7,365.69,0.44,3
8,Hyperliquid,AAVE,1m,1737517140000,1737517199999,AAVE,1m,366.73,366.94,366.94,366.73,16.61,13
9,NaN,NaN,NaN,1737517200000,1737517259999,AAVE,1m,367.66,367.66,367.66,367.66,1.4,1
